In [137]:
import re, subprocess, boto3, json, shlex, mysql, os, urllib
import pandas as pd
import numpy as np
from s3path import S3Path
from pathlib import Path
from tqdm.notebook import tqdm
from packaging import version
pd.set_option("display.max_colwidth", 40)

In [138]:
from utils.utils import *

# Process Samples

## Setup

In [107]:
# Location of docker files
common_docker_registry = "quay.io/hisplan"

prefix = "CellRangerArc" # Workflow to run; also .wdl filename prefix
pipeline_type = prefix # field in *.labels.json
output_dirname = "cr-arc-results"

# If need to add comment, put here
comment = ""

In [108]:
# Locations of workflow-related directories and files
path_to_cromwell_secrets = f"{Path.home()}/.cromwell/cromwell-secrets.json" # CHANGE THIS
workflow_dir = f"{Path.home()}/scing/bin/cellranger-arc-2.0.0" # CHANGE THIS
path_to_exec = f"{workflow_dir}/submit.sh" # CHANGE THIS FOR SHARP
config_dir = f"{workflow_dir}/configs"
path_to_options = f"{workflow_dir}/{prefix}.options.aws.json"

# Other file locations
db_credentials_path = f"{Path.home()}/.config.json" # CHANGE THIS

In [109]:
# Set credentials based on SCRIdb CLI config file
with open(db_credentials_path) as f:
    creds = json.load(f)

In [110]:
# Samples on which to run CellRangerATAC
# Note: Assumes data is transferred to AWS S3 (this should be an s3 location)
# Note: Assumes directory name is name of sample
common_dir = "s3://dp-lab-data/collaborators/VanDenBrink/AgingHematopoiesis"
samples = [
    "HA-1536_Young_mice_multiome",
    "HA-1536_Old_mice_multiome"
]
sample_paths = [
    f"{common_dir}/{sample}" for sample in samples
]

## Execution

In [130]:
# Get information for all samples
sample_paths = [s.strip('/') for s in sample_paths] # remove trailing slash if exists
sample_names = [os.path.basename(s) for s in sample_paths]
#sample_names = [s.replace("Redo_", "") for s in sample_names]
print(sample_names)
samples = pd.DataFrame(
    sample_paths,
    index=sample_names,
    columns=["S3_Path"],
    dtype=str,
)
samples["Sample_ID"] = pd.Series(samples.index).apply(
    lambda x: get_sample_id(x, creds['user'], creds['password'])
).values

# Get FASTQ paths from S3
# Note: Uses same FASTQ file ids for all samples
#fastq_file_ids = fastq_map[prefix]
samples["FASTQs_GEX"] = samples["S3_Path"].apply(lambda x: get_fastqs(x, folder="FASTQ"))
samples["FASTQs_ATAC"] = samples["S3_Path"].apply(lambda x: get_fastqs(x + "_ATAC", folder="FASTQ"))

# Get reference genome location
samples["Reference"] = samples["Sample_ID"].apply(lambda x: get_cr_reference(x, prefix, creds["user"], creds["password"]))

['HA-1536_Young_mice_multiome', 'HA-1536_Old_mice_multiome']


In [141]:
# Load minimum inputs and labels fields from templates
with open(f"{config_dir}/template.inputs.json") as f:
    std_inputs_fields = list(json.load(f).keys())
    
with open(f"{config_dir}/template.labels.json") as f:
    std_labels_fields = list(json.load(f).keys())
    
# Annotate all samples with workflow inputs and labels
inputs = pd.DataFrame(index=samples.index, columns=std_inputs_fields,)
labels = pd.DataFrame(index=samples.index, columns=std_labels_fields,)

# Annotate inputs
inputs[f"{prefix}.runID"] = inputs.index # may need to change
inputs[f"{prefix}.gexFastqFiles"] = samples["FASTQs_GEX"].apply(lambda x: np.ravel(list(x.values())))
inputs[f"{prefix}.gexFastqName"] = inputs[f"{prefix}.gexFastqFiles"].apply(lambda x: get_fastqs_name(x))
inputs[f"{prefix}.atacFastqFiles"] = samples["FASTQs_ATAC"].apply(lambda x: np.ravel(list(x.values())))
inputs[f"{prefix}.atacFastqName"] = inputs[f"{prefix}.atacFastqFiles"].apply(lambda x: get_fastqs_name(x))
inputs[f"{prefix}.reference"] = samples["Reference"]                                 
inputs[f"{prefix}.dockerRegistry"] = common_docker_registry

# Annotate labels
labels["pipelineType"] = "CellRangerARC"
labels["project"] = samples["Sample_ID"].apply(lambda x: get_project_id(x, creds["user"], creds["password"]))
labels["sample"] = labels.index
labels["owner"] = creds["user"]
labels["destination"] = samples['S3_Path'] + "/" + output_dirname
labels["transfer"] = "-"
labels["comment"] = creds["user"]

assert (std_inputs_fields == list(inputs.columns)) & (inputs.notna().values.all())
assert (std_labels_fields == list(labels.columns)) & (labels.notna().values.all())

In [142]:
inputs

,CellRangerArc.runID,CellRangerArc.gexFastqName,CellRangerArc.gexFastqFiles,CellRangerArc.atacFastqName,CellRangerArc.atacFastqFiles,CellRangerArc.reference,CellRangerArc.dockerRegistry
HA-1536_Young_mice_multiome,HA-1536_Young_mice_multiome,3573_HA-1536_Young_mice_multiome_IGO...,[s3://dp-lab-data/collaborators/VanD...,3575_HA-1536_Young_mice_multiome_ATA...,[s3://dp-lab-data/collaborators/VanD...,https://cf.10xgenomics.com/supp/cell...,quay.io/hisplan
HA-1536_Old_mice_multiome,HA-1536_Old_mice_multiome,3574_HA-1536_Old_mice_multiome_IGO_1...,[s3://dp-lab-data/collaborators/VanD...,3576_HA-1536_Old_mice_multiome_ATAC_...,[s3://dp-lab-data/collaborators/VanD...,https://cf.10xgenomics.com/supp/cell...,quay.io/hisplan


In [143]:
labels

,pipelineType,project,sample,owner,destination,transfer,comment
HA-1536_Young_mice_multiome,CellRangerARC,Aging hematopoiesis,HA-1536_Young_mice_multiome,moormana,s3://dp-lab-data/collaborators/VanDe...,-,moormana
HA-1536_Old_mice_multiome,CellRangerARC,Aging hematopoiesis,HA-1536_Old_mice_multiome,moormana,s3://dp-lab-data/collaborators/VanDe...,-,moormana


In [139]:
stdouts = [] # to store all outputs
process = True

with tqdm(samples.index) as t:

    for sample_name in t:

        # Write inputs and labels to file
        path_to_inputs = f"{config_dir}/{sample_name}.inputs.json"
        with open(path_to_inputs, "w") as f_inputs:
            json.dump(inputs.loc[sample_name].to_dict(), f_inputs, indent=4, cls=NpEncoder)

        path_to_labels = f"{config_dir}/{sample_name}.labels.json"
        with open(path_to_labels, "w") as f_labels:
            json.dump(labels.loc[sample_name].to_dict(), f_labels, indent=4, cls=NpEncoder)

        if process:
            stdouts.append(run(
                workflow_path = workflow_dir,
                execp = "submit.sh",
                secrets = path_to_cromwell_secrets,
                inputs = path_to_inputs,
                labels = path_to_labels,
                options = path_to_options,
            ))

  0%|          | 0/2 [00:00<?, ?it/s]

In [140]:
stdouts

[{'args': ['/Users/moormana/scing/bin/cellranger-arc-2.0.0/submit.sh',
   '-k',
   '/Users/moormana/.cromwell/cromwell-secrets.json',
   '-i',
   '/Users/moormana/scing/bin/cellranger-arc-2.0.0/configs/HA-1536_Young_mice_multiome.inputs.json',
   '-l',
   '/Users/moormana/scing/bin/cellranger-arc-2.0.0/configs/HA-1536_Young_mice_multiome.labels.json',
   '-o',
   '/Users/moormana/scing/bin/cellranger-arc-2.0.0/CellRangerArc.options.aws.json'],
  'returncode': 0,
  'stdout': '{"id":"bcd07dca-0c91-48a3-99d8-962bf33608b3","status":"Submitted"}\n',
  'stderr': ''},
 {'args': ['/Users/moormana/scing/bin/cellranger-arc-2.0.0/submit.sh',
   '-k',
   '/Users/moormana/.cromwell/cromwell-secrets.json',
   '-i',
   '/Users/moormana/scing/bin/cellranger-arc-2.0.0/configs/HA-1536_Old_mice_multiome.inputs.json',
   '-l',
   '/Users/moormana/scing/bin/cellranger-arc-2.0.0/configs/HA-1536_Old_mice_multiome.labels.json',
   '-o',
   '/Users/moormana/scing/bin/cellranger-arc-2.0.0/CellRangerArc.options.